In [1]:
# Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

In [13]:
# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 8000)")
    time.sleep(1)
scroll_down(driver)

In [8]:
# Step 1. 크롬 웹브라우저 실행
# driver = webdriver.Chrome("/Users/aiden/Desktop/chromedriver")  # 윈도우는 "chromedriver.exe"
# url = "https://www.booking.com/searchresults.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=e19b301d9f2ba429173c4b6aaa971bcd&tmpl=searchresults&checkin_month=5&checkin_monthday=15&checkin_year=2021&checkout_month=5&checkout_monthday=16&checkout_year=2021&class_interval=1&dest_id=4170&dest_type=region&dtdisc=0&from_sf=1&group_adults=1&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=region&room1=A&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=index&src_elem=sb&srpvid=d60890e0e8ae0830&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84%2C%20%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ss_all=0&ssb=empty&sshis=0&top_ufis=1&rows=25\
# &offset=0"
# driver.get(url)
# time.sleep(2)

# 데이터 크롤링

In [66]:
# Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome("/Users/aiden/Desktop/chromedriver")  # 윈도우는 "chromedriver.exe"
url = "https://www.booking.com/searchresults.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=e19b301d9f2ba429173c4b6aaa971bcd&tmpl=searchresults&checkin_month=5&checkin_monthday=15&checkin_year=2021&checkout_month=5&checkout_monthday=16&checkout_year=2021&class_interval=1&dest_id=4170&dest_type=region&dtdisc=0&from_sf=1&group_adults=1&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=region&room1=A&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=index&src_elem=sb&srpvid=d60890e0e8ae0830&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84%2C%20%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ss_all=0&ssb=empty&sshis=0&top_ufis=1&rows=25\
&offset=0"
driver.get(url)
time.sleep(2)

hotel_list = []
url_list=[]
rating_list = []
price_list = []
room_list = []

print("크롤링 성공: ")
print('hotel', 'rating', 'price', 'room', 'url')

page = 5   # 크롤링 원하는 페이지 수
for i in range(0, page):
    
    # 호텔 이름
    hotel_raw = driver.find_elements_by_css_selector('.sr-hotel__name')
    for hotel in hotel_raw:
        hotel = hotel.text
        hotel_list.append(hotel)
    # hotel_list = [ x for x in hotel_list if not x =='']
    
    # 별점 가져오기
    rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
    for rating in rating_raw:
        if '\n' in rating.text:
            rating = float(rating.text.split('\n')[0])
        else:
            rating = float(rating.text)
        rating_list.append(rating)
    
    # 가격 가져오기
    price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
    for price in price_raw:
        if price.text.split('\n') != ['']:
            price = price.text.split('\n')
            price = int([ x for x in price if x != '1박, 성인 1명'][0].split(' ')[-1].replace('₩', '').replace(',', ''))
            price_list.append(price)
    price_list = [x for x in price_list if not x =='']
    
    # 방 정보
    room_raw = driver.find_elements_by_css_selector(".room_link")
    for room in room_raw:
        room_list.append(room.text.split('\n')[0].replace(' -','').strip())
    room_list = [ x for x in room_list if not x =='']
    
    # url
    url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")
    for url in url_raw:
        url = url.get_attribute('href')
        url_list.append(url)
    time.sleep(1)
    # url_list = [ x for x in url_list if not x =='']
    
    # 크롤링 성공한 갯수
    print(len(hotel_list),len(rating_list),len(price_list),len(room_list),len(url_list))
    
    # 다음 페이지 이동 - 페이지 이동에 관해서는 지연 시간을 넉넉하게 주는것이 좋다
    driver.find_element_by_css_selector('.bk-icon.-iconset-navarrow_right.bui-pagination__icon').click()
    time.sleep(3)
    
print("크롤링 완료")

크롤링 성공: 
hotel rating price room url
25 25 25 25 25
50 50 50 50 50
75 75 75 75 75
100 100 100 100 100
125 125 125 125 125
크롤링 완료


In [67]:
print(len(hotel_list))
print(len(room_list))
print(len(price_list))
print(len(rating_list))
print(len(url_list))

125
125
125
125
125


In [68]:
# 데이터 프레임으로 만들기
# 행의 개수가 일치 해야한다.
df = pd.DataFrame({'hotel':hotel_list, 'rating':rating_list, 'room':room_list, 'url':url_list, 'price(won)':price_list})
df    

,hotel,rating,room,url,price(won)
0,신신호텔 서귀포,8.6,Standard Ondol Room(Single use),https://www.booking.com/hotel/kr/sinsinhotel-s...,43934
1,롯데호텔 제주,8.6,한실 패밀리룸,https://www.booking.com/hotel/kr/lotte-jeju.ko...,514552
2,랜딩관 제주신화월드 호텔앤리조트,8.3,수페리어 트윈룸,https://www.booking.com/hotel/kr/jeju-shinhwa-...,220000
3,서머셋 제주신화월드,8.6,패밀리 스위트룸 한실 (온돌),https://www.booking.com/hotel/kr/somerset-jeju...,660000
4,메종 글래드 제주,8.1,디럭스 트윈룸,https://www.booking.com/hotel/kr/jeju-grand.ko...,175500
...,...,...,...,...,...
120,더제주리조트,6.5,스탠다드 트윈룸,https://www.booking.com/hotel/kr/benikea-the-j...,40000
121,산방산애 리조트,8.4,2베드룸 스위트,https://www.booking.com/hotel/kr/sanbangsanae-...,158193
122,아라팰리스 호텔,7.5,퀸룸 바다 전망,https://www.booking.com/hotel/kr/arapaelriseuh...,72000
123,제주 길라임 펜션,8.7,싱글룸,https://www.booking.com/hotel/kr/jeju-gillime-...,28350


In [69]:
# 엑셀로 저장하기
df.to_excel("booking_com.xlsx", encoding='utf-8-sig')

# (참고 코드)

In [14]:
# 호텔 이름
hotel_raw = driver.find_elements_by_css_selector('.sr-hotel__name')
hotel_list = []
for hotel in hotel_raw:
    hotel = hotel.text
    hotel_list.append(hotel)
    
# hotel_list = [ x for x in hotel_list if not x =='']
print(len(hotel_list))
hotel_list

25


['신신호텔 서귀포',
 '롯데호텔 제주',
 '서머셋 제주신화월드',
 '메종 글래드 제주',
 '신화관 제주신화월드 호텔 앤 리조트',
 '메리어트관 제주신화월드 호텔앤리조트',
 '그랜드 하얏트 제주',
 '롯데시티호텔 제주',
 '브라운 스위트 호텔 & 리조트',
 '호텔 휘슬락',
 '라마다 프라자 호텔',
 '호텔 리젠트 마린 더블루',
 '호텔난타 제주',
 '코업시티호텔 하버뷰',
 '해비치 호텔 & 리조트',
 '골든 데이지 호텔',
 '퍼스트70 호텔',
 '그랜드 조선 제주',
 '더 그랜드 섬오름',
 '휘닉스 제주 섭지코지',
 '코업시티호텔 성산',
 '제주 센트럴 시티호텔',
 '제주 마레보 비치 호텔',
 '위 호텔 제주',
 '신신호텔 제주공항']

In [15]:
# url 
url_list=[]
url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")

for url in url_raw:
    url = url.get_attribute('href')
    url_list.append(url)
    
# url_list = [ x for x in url_list if not x =='']
print(len(url_list))
url_list

25


['https://www.booking.com/hotel/kr/sinsinhotel-seogwipo.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=11607fb15db3223ead36f4cd0a8c3778&all_sr_blocks=696587412_287464310_1_2_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4170&dest_type=region&group_adults=1&group_children=0&hapos=1&highlighted_blocks=696587412_287464310_1_2_0&hpos=1&no_rooms=1&req_adults=1&req_children=0&room1=A&sr_order=popularity&sr_pri_blocks=696587412_287464310_1_2_0__4393400&srepoch=1620698585&srpvid=db600e6c104600f9&ucfs=1&from=searchresults&tpi_r=2#hotelTmpl',
 'https://www.booking.com/hotel/kr/lotte-jeju.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=11607fb15db3223ead36f4cd0a8c3778&all_sr_blocks=23513931_235897805_2_42_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4

In [34]:
url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")
url_raw[0].get_attribute('href')

'https://www.booking.com/hotel/kr/noblesse-jeju.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=a8969e8a7e83a3dfff572abb1c6df87a&all_sr_blocks=148081801_114185930_2_0_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4170&dest_type=region&group_adults=1&group_children=0&hapos=251&highlighted_blocks=148081801_114185930_2_0_0&hpos=1&no_rooms=1&req_adults=1&req_children=0&room1=A&sr_order=popularity&sr_pri_blocks=148081801_114185930_2_0_0__5400000&srepoch=1620699854&srpvid=b14010e61385012c&ucfs=1&from=searchresults&tpi_r=2#hotelTmpl'

In [62]:
# 별점 가져오기
rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
rating_list = []
for rating in rating_raw:
    if '\n' in rating.text:
        rating = float(rating.text.split('\n')[0])
    else:
        rating = float(rating.text)
    rating_list.append(rating)
    
# rating_list = [ x for x in rating_list if not x =='']
print(len(rating_list))
rating_list

25


[8.2,
 7.8,
 6.8,
 7.1,
 9.0,
 9.4,
 8.5,
 8.7,
 8.5,
 9.0,
 8.9,
 9.0,
 7.8,
 7.7,
 7.8,
 8.3,
 8.6,
 7.8,
 5.5,
 8.0,
 8.8,
 8.2,
 7.8,
 6.7,
 9.0]

In [44]:
rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
for i in rating_raw:    
    print(i.text, '\n')

6.6
이용 후기 평점
13개 이용 후기
새창에서 열기 

8.2
매우 좋음
160개 이용 후기
새창에서 열기 

7.2
좋음
13개 이용 후기
새창에서 열기 

6.9
이용 후기 평점
9개 이용 후기
새창에서 열기 

9.8
강력 추천
155개 이용 후기
새창에서 열기
위치 9.6
새창에서 열기 

8.0
매우 좋음
159개 이용 후기
새창에서 열기 

8.4
매우 좋음
5개 이용 후기
새창에서 열기
위치 9.5
새창에서 열기 

6.6
이용 후기 평점
51개 이용 후기
새창에서 열기 

7.8
좋음
31개 이용 후기
새창에서 열기 

6.7
이용 후기 평점
10개 이용 후기
새창에서 열기 

8.1
매우 좋음
92개 이용 후기
새창에서 열기 

7.9
좋음
12개 이용 후기
새창에서 열기 

8.6
우수함
28개 이용 후기
새창에서 열기 

8.1
매우 좋음
20개 이용 후기
새창에서 열기 

9.4
최고
113개 이용 후기
새창에서 열기 

 

7.5
좋음
10개 이용 후기
새창에서 열기 

8.2
매우 좋음
142개 이용 후기
새창에서 열기 

9.0
최고
10개 이용 후기
새창에서 열기 

7.8
좋음
14개 이용 후기
새창에서 열기 

8.1
매우 좋음
7개 이용 후기
새창에서 열기 

7.6
좋음
12개 이용 후기
새창에서 열기 

6.6
이용 후기 평점
45개 이용 후기
새창에서 열기 

7.6
좋음
7개 이용 후기
새창에서 열기 

7.3
좋음
22개 이용 후기
새창에서 열기 



In [51]:
# 가격 가져오기
price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
price_list = []
for price in price_raw:
    if price.text.split('\n') != ['']:
        price = price.text.split('\n')
        price = int([ x for x in price if x != '1박, 성인 1명'][0].split(' ')[-1].replace('₩', '').replace(',', ''))
        price_list.append(price)
price_list = [x for x in price_list if not x ==''] # 리스트 안에 특정요소 제거하고 객체 저장
print(len(price_list))
price_list

25


[54000,
 23037,
 60450,
 138225,
 20273,
 46075,
 35000,
 53550,
 40000,
 49761,
 44232,
 82935,
 25000,
 35000,
 25000,
 65427,
 50682,
 49000,
 72799,
 58054,
 103669,
 50775,
 35000,
 50000,
 23498]

In [46]:
price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
for i in price_raw:
    print(i.text.split('\n'))    

['1박, 성인 1명', '₩54,000', '요금 ₩54,000', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩25,000 ₩23,037', '기존 요금 ₩25,000 현재 요금 ₩23,037', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩80,108 ₩60,450', '기존 요금 ₩80,108 현재 요금 ₩60,450', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩150,000 ₩138,225', '기존 요금 ₩150,000 현재 요금 ₩138,225', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩22,000 ₩20,273', '기존 요금 ₩22,000 현재 요금 ₩20,273', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩50,000 ₩46,075', '기존 요금 ₩50,000 현재 요금 ₩46,075', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩35,000', '요금 ₩35,000', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩53,550', '요금 ₩53,550', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩40,000', '요금 ₩40,000', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩54,000 ₩49,761', '기존 요금 ₩54,000 현재 요금 ₩49,761', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩80,211 ₩44

In [18]:
# 방 정보
room_raw = driver.find_elements_by_css_selector(".room_link")
room_list = []
for room in room_raw:
    room_list.append(room.text.split('\n')[0].replace(' -','').strip()) #strip() - 문자열 데이터 양 옆에 빈공간 제거
    
room_list = [ x for x in room_list if not x =='']
print(len(room_list))
room_list

25


['Standard Ondol Room(Single use)',
 '한실 패밀리룸',
 '패밀리 스위트룸 한실 (온돌)',
 '디럭스 트윈룸',
 '수페리어 트윈룸',
 '패밀리룸',
 '객실 킹침대 1개, 도시 전망',
 '스테이케이션 혜택 스탠다드 트윈룸 14:00에 늦은 체크아웃',
 '스탠다드 트윈룸',
 '베이직 옵션',
 '수페리어 트윈룸 산 전망',
 '디럭스 더블룸 바다 전망',
 '디럭스 트윈룸',
 '마운틴 뷰 더블룸',
 '[리조트] 스튜디오 트윈 빌리지',
 '수페리어 트윈',
 '수페리어 트윈룸',
 '디럭스 트윈룸 산의 전망',
 '수페리어 더블룸 신관',
 '로얄룸',
 '스탠다드 더블룸',
 '수페리어 더블룸',
 '디럭스 더블룸 바다 전망',
 '수페리어 트윈룸 바다 전망',
 '체크인 시 객실 배정 주차 불가']